In [1]:
import pickle
import h5py
import os
from sklearn import svm
from statistics import mean
from random import shuffle
import numpy as np
from sklearn.utils import shuffle as shf
from sklearn.neural_network import MLPClassifier

In [2]:
data2 = {}
print('loading data')
for key in os.listdir('E:/Time Series/hdf_files'):
	filename = 'E:/Time Series/hdf_files/'+ key
	f = h5py.File(filename, 'r')
	data2[key] = list(list(f.values())[0])
	f.close()

loading data


In [3]:
data = data2

In [4]:
for key, value in data.items():
    print(key, '---', len(value))
    shuffle(data[key])

ALFALFA --- 20180
CORN --- 1753433
DEVELOPED --- 235966
FOREST --- 204529
OATS --- 2157
RYE --- 109
SOYABEENS --- 1580198
WATER_BODY --- 25743


In [5]:
data = {key:value[:20000] for key, value in data.items() if len(value) >= 20000}

In [17]:
for key in data.keys():
    for n in range(len(data[key])):
        example = list(data[key][n])
        for i in range(45):
            example.append(example[i]*example[i])
        data[key][n] = example

C:\Users\prabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in ushort_scalars
  """


In [6]:
mins = {}; maxs = {}; means = {}
for i in range(45):
    lin = [l[i] for value in data.values() for l in value]
    mins[i] = min(lin)
    maxs[i] = max(lin)
    means[i] = mean(lin)
    
data = {key: [[np.float64((example[i]-means[i])/(maxs[i] - mins[i])) for i in range(45)] for example in value] for key, value in data.items()}

C:\Users\prabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in ushort_scalars
  


In [7]:
data3 = data #basic data

In [19]:
data4 = data #with quadratic terms

In [8]:
data = data3

In [9]:
for key in data.keys():
    shuffle(data[key])

In [22]:
training_examples = []
training_labels = []
test_examples = []
test_labels = []
cv_examples = []
cv_labels = []
i = 0;
for key, value in data.items():
    training_examples = training_examples + value[:18000]
    cv_examples = cv_examples + value[18000:19000]
    test_examples = test_examples + value[19000:]
    training_labels = training_labels + [i for v in range(18000)]
    cv_labels = cv_labels + [i for v in range(1000)]
    test_labels = test_labels + [i for v in range(1000)]
    print(i,':', key)
    i+=1

0 : ALFALFA
1 : CORN
2 : DEVELOPED
3 : FOREST
4 : SOYABEENS
5 : WATER_BODY


In [23]:
training_examples, training_labels  = shf(training_examples+cv_examples, training_labels+cv_labels)

In [24]:
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(training_examples, training_labels)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [25]:
s = model.feature_importances_
L = np.array(s)
t = sorted(range(len(L)), key=lambda i:L[i])
z = []; r = 0
for j in t[::-1]:
    print( j, '-----', s[j])
    if s[j]>0.019: z.append(j)

44 ----- 0.05689138342908255
34 ----- 0.044961685771263665
39 ----- 0.04445599606430319
4 ----- 0.04336965191784822
40 ----- 0.040217375392934206
15 ----- 0.03343321257797839
14 ----- 0.03312164965007875
24 ----- 0.03234844713744776
10 ----- 0.031109301369145505
30 ----- 0.030324439276916394
29 ----- 0.024572590135006546
0 ----- 0.024159223188578306
19 ----- 0.023620707522889077
37 ----- 0.02225816152045502
2 ----- 0.02225671151652781
5 ----- 0.021294964626234984
25 ----- 0.021027147984044554
1 ----- 0.021008694150807847
12 ----- 0.020488233003941904
9 ----- 0.02025907833310559
23 ----- 0.018795116450663847
7 ----- 0.018513992354528816
17 ----- 0.018398574208920437
22 ----- 0.018214200295697935
26 ----- 0.01816420536339443
20 ----- 0.017931315438434577
35 ----- 0.017619148486262986
27 ----- 0.01685447587184389
36 ----- 0.01668565262310511
11 ----- 0.016467602229886025
42 ----- 0.016453445272418633
32 ----- 0.01591047528966951
13 ----- 0.015237730426668733
43 ----- 0.015008101983340072


In [86]:
data = {key:[[example[i] for i in z]for example in value] for key, value in data.items()}
print(len(z))

22


In [10]:
training_examples = []
training_labels = []
test_examples = []
test_labels = []
cv_examples = []
cv_labels = []
i = 0;
for key, value in data.items():
    training_examples = training_examples + value[:18000]
    cv_examples = cv_examples + value[18000:19000]
    test_examples = test_examples + value[19000:]
    training_labels = training_labels + [i for v in range(18000)]
    cv_labels = cv_labels + [i for v in range(1000)]
    test_labels = test_labels + [i for v in range(1000)]
    print(i,':', key)
    i+=1

0 : ALFALFA
1 : CORN
2 : DEVELOPED
3 : FOREST
4 : SOYABEENS
5 : WATER_BODY


In [11]:
training_examples, training_labels  = shf(training_examples, training_labels)

In [28]:
clf = svm.SVC(C=100000,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.0000001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)


In [ ]:
clf.fit(training_examples, training_labels)

[LibSVM]

In [152]:
sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels)

0.714

In [153]:
sum(clf.predict(training_examples) == training_labels)/len(training_labels)

0.87416

In [144]:
for i in clf.predict(training_examples):
    if i == 5:print (i)

In [145]:
import sklearn
sklearn.metrics.log_loss(training_labels, clf.predict(training_examples), eps=1e-15, normalize=True, sample_weight=None, labels=None)

ValueError: y_true and y_pred contain different number of classes 5, 2. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [0 1 2 3 4]

In [ ]:
sklearn.metrics.log_loss(cv_labels, clf.predict(cv_labels), eps=1e-15, normalize=True, sample_weight=None, labels=None)

In [ ]:
sum(clf.predict(test_examples) == test_labels)/len(test_labels)

In [61]:
file_object = open('clf.pkl', 'wb')
pickle.dump(clf, file_object)
file_object.close()

In [36]:
cnf = MLPClassifier(hidden_layer_sizes=(30,30,10,10),
                    activation='relu', 
                    solver='sgd',
                    alpha=0.000001, 
                    batch_size= 100000, 
                    learning_rate='adaptive',
                    learning_rate_init=0.001, 
                    power_t=0.5, 
                    max_iter=10000, 
                    shuffle=True, 
                    random_state=None, 
                    tol=0.0000001, 
                    verbose=True, 
                    warm_start=False, 
                    momentum=0.9, 
                    nesterovs_momentum=True, 
                    early_stopping=False,
                    validation_fraction=0.1, 
                    beta_1=0.9, 
                    beta_2=0.999, 
                    epsilon=1e-08
                   )

In [ ]:
cnf.fit(training_examples, training_labels)

Iteration 1, loss = 1.96151174
Iteration 2, loss = 1.94649728
Iteration 3, loss = 1.92505803
Iteration 4, loss = 1.90125697
Iteration 5, loss = 1.87672548
Iteration 6, loss = 1.85668903
Iteration 7, loss = 1.84128531
Iteration 8, loss = 1.82968394
Iteration 9, loss = 1.82137371
Iteration 10, loss = 1.81417123
Iteration 11, loss = 1.80619709
Iteration 12, loss = 1.79742230
Iteration 13, loss = 1.78789487
Iteration 14, loss = 1.77824058
Iteration 15, loss = 1.76874930
Iteration 16, loss = 1.75931971
Iteration 17, loss = 1.74972688
Iteration 18, loss = 1.74022370
Iteration 19, loss = 1.73094964
Iteration 20, loss = 1.71928100
Iteration 21, loss = 1.70401699
Iteration 22, loss = 1.68657378
Iteration 23, loss = 1.66966593
Iteration 24, loss = 1.66251995
Iteration 25, loss = 1.65391622
Iteration 26, loss = 1.64429217
Iteration 27, loss = 1.63579094
Iteration 28, loss = 1.62890851
Iteration 29, loss = 1.62239394
Iteration 30, loss = 1.61623099
Iteration 31, loss = 1.61044056
Iteration 32, los

Iteration 253, loss = 1.37269622
Iteration 254, loss = 1.37244989
Iteration 255, loss = 1.37220669
Iteration 256, loss = 1.37195451
Iteration 257, loss = 1.37171915
Iteration 258, loss = 1.37147049
Iteration 259, loss = 1.37122934
Iteration 260, loss = 1.37099308
Iteration 261, loss = 1.37073860
Iteration 262, loss = 1.37049724
Iteration 263, loss = 1.37027764
Iteration 264, loss = 1.37002968
Iteration 265, loss = 1.36979979
Iteration 266, loss = 1.36956946
Iteration 267, loss = 1.36933834
Iteration 268, loss = 1.36910366
Iteration 269, loss = 1.36886948
Iteration 270, loss = 1.36864090
Iteration 271, loss = 1.36841072
Iteration 272, loss = 1.36818455
Iteration 273, loss = 1.36796270
Iteration 274, loss = 1.36773657
Iteration 275, loss = 1.36751403
Iteration 276, loss = 1.36729166
Iteration 277, loss = 1.36707143
Iteration 278, loss = 1.36685123
Iteration 279, loss = 1.36661785
Iteration 280, loss = 1.36639245
Iteration 281, loss = 1.36617204
Iteration 282, loss = 1.36595184
Iteration 

Iteration 502, loss = 1.33523622
Iteration 503, loss = 1.33513813
Iteration 504, loss = 1.33504293
Iteration 505, loss = 1.33495708
Iteration 506, loss = 1.33486082
Iteration 507, loss = 1.33475888
Iteration 508, loss = 1.33465924
Iteration 509, loss = 1.33457808
Iteration 510, loss = 1.33447347
Iteration 511, loss = 1.33438644
Iteration 512, loss = 1.33428675
Iteration 513, loss = 1.33419407
Iteration 514, loss = 1.33409579
Iteration 515, loss = 1.33400647
Iteration 516, loss = 1.33390021
Iteration 517, loss = 1.33380556
Iteration 518, loss = 1.33371649
Iteration 519, loss = 1.33362793
Iteration 520, loss = 1.33353152
Iteration 521, loss = 1.33343708
Iteration 522, loss = 1.33334798
Iteration 523, loss = 1.33325729
Iteration 524, loss = 1.33316829
Iteration 525, loss = 1.33308356
Iteration 526, loss = 1.33299641
Iteration 527, loss = 1.33289994
Iteration 528, loss = 1.33280581
Iteration 529, loss = 1.33271376
Iteration 530, loss = 1.33262990
Iteration 531, loss = 1.33255230
Iteration 

Iteration 751, loss = 1.31679173
Iteration 752, loss = 1.31672849
Iteration 753, loss = 1.31667545
Iteration 754, loss = 1.31663067
Iteration 755, loss = 1.31657227
Iteration 756, loss = 1.31651483
Iteration 757, loss = 1.31646270
Iteration 758, loss = 1.31639793
Iteration 759, loss = 1.31635213
Iteration 760, loss = 1.31628926
Iteration 761, loss = 1.31623025
Iteration 762, loss = 1.31617713
Iteration 763, loss = 1.31611301
Iteration 764, loss = 1.31606770
Iteration 765, loss = 1.31600181
Iteration 766, loss = 1.31594399
Iteration 767, loss = 1.31588965
Iteration 768, loss = 1.31586595
Iteration 769, loss = 1.31579149
Iteration 770, loss = 1.31573586
Iteration 771, loss = 1.31567989
Iteration 772, loss = 1.31562122
Iteration 773, loss = 1.31556281
Iteration 774, loss = 1.31550478
Iteration 775, loss = 1.31544634
Iteration 776, loss = 1.31539433
Iteration 777, loss = 1.31533436
Iteration 778, loss = 1.31529260
Iteration 779, loss = 1.31523206
Iteration 780, loss = 1.31517114
Iteration 

Iteration 1000, loss = 1.30523919
Iteration 1001, loss = 1.30521862
Iteration 1002, loss = 1.30516397
Iteration 1003, loss = 1.30512956
Iteration 1004, loss = 1.30508977
Iteration 1005, loss = 1.30505199
Iteration 1006, loss = 1.30500991
Iteration 1007, loss = 1.30497353
Iteration 1008, loss = 1.30493240
Iteration 1009, loss = 1.30490552
Iteration 1010, loss = 1.30486109
Iteration 1011, loss = 1.30482276
Iteration 1012, loss = 1.30479556
Iteration 1013, loss = 1.30475861
Iteration 1014, loss = 1.30474972
Iteration 1015, loss = 1.30469635
Iteration 1016, loss = 1.30465669
Iteration 1017, loss = 1.30460922
Iteration 1018, loss = 1.30456587
Iteration 1019, loss = 1.30452182
Iteration 1020, loss = 1.30448772
Iteration 1021, loss = 1.30444421
Iteration 1022, loss = 1.30440715
Iteration 1023, loss = 1.30436905
Iteration 1024, loss = 1.30433213
Iteration 1025, loss = 1.30429965
Iteration 1026, loss = 1.30426103
Iteration 1027, loss = 1.30423784
Iteration 1028, loss = 1.30418475
Iteration 1029

Iteration 1241, loss = 1.29719575
Iteration 1242, loss = 1.29716759
Iteration 1243, loss = 1.29714538
Iteration 1244, loss = 1.29712447
Iteration 1245, loss = 1.29712150
Iteration 1246, loss = 1.29708906
Iteration 1247, loss = 1.29704617
Iteration 1248, loss = 1.29704297
Iteration 1249, loss = 1.29697792
Iteration 1250, loss = 1.29694180
Iteration 1251, loss = 1.29691348
Iteration 1252, loss = 1.29688371
Iteration 1253, loss = 1.29686033
Iteration 1254, loss = 1.29682551
Iteration 1255, loss = 1.29679731
Iteration 1256, loss = 1.29677284
Iteration 1257, loss = 1.29674625
Iteration 1258, loss = 1.29674524
Iteration 1259, loss = 1.29669892
Iteration 1260, loss = 1.29666288
Iteration 1261, loss = 1.29663298
Iteration 1262, loss = 1.29660623
Iteration 1263, loss = 1.29657300
Iteration 1264, loss = 1.29654257
Iteration 1265, loss = 1.29652092
Iteration 1266, loss = 1.29649189
Iteration 1267, loss = 1.29645795
Iteration 1268, loss = 1.29642741
Iteration 1269, loss = 1.29640346
Iteration 1270

In [ ]:
sum(cnf.predict(cv_examples) == cv_labels)/len(cv_labels)

In [ ]:
sum(cnf.predict(training_examples) == training_labels)/len(training_labels)

In [ ]:
sum(cnf.predict(test_examples) == test_labels)/len(test_labels)